In [124]:
#rush hour game methods definition

import numpy as np
#based on gameboard and car letter return x or y
def moveAxis(gameboard,car):
    
    for char in range (len(gameboard)):
        if gameboard[char]==car and 'p1' not in locals():
            p1=char
        elif gameboard[char]==car:  
            p2=char
            break
    if (p1-p2)==-6:
        return 'y'
    if (p1-p2)==-1:
        return 'x'

    
#read text file and return  6x6 array of gameboard and dict of fuel level 
def readfile(filename):
    gameboards=[]
    carFuels=[]
    f = open(filename, "rt")
    count=0
    for x in f:
        gameboard=[]
        carFuel={}
        if x[0]=="#":
            continue
        if len(x)==1:
            continue
        else:
            x=x.strip("\n")
            token=x.split()
            
            for i in range(len(token)):        
                if i==0:
                    count+=1
                    for j in range(36):
                        gameboard.insert(len(gameboard),token[i][j])
                        if token[i][j] not in  carFuel and  token[i][j] !='.':
                            carFuel[token[i][j]]=100
                elif i>=1: 
                    
                    car=token[i][0]
                    fuelLevel=int(token[i][1])
                    carFuel.update({car:fuelLevel})
        gameboard=np.array(gameboard).reshape(6,6)
        gameboards.insert(count-1,gameboard)
        carFuels.insert(count-1,carFuel)        
                          
    f.close()
    return gameboards,carFuels
#adds the valet service to the game
def valetService(gameboard):
    if gameboard[17]!=".":
        car=gameboard[17]
        gameboard[:] = [x if x != car else "." for x in gameboard]
    return gameboard    
    
#makes a dict of possible moves
def possiblemoves(gameboard,fuelLevel):
    gameboard=gameboard.flatten().tolist()
    moves={}
    
    for car in fuelLevel.keys():
        if car in gameboard:
            axis=moveAxis(gameboard,car)
            carMoves={}   
            if(axis=='x'):       
                left=MovePossibleLeft(gameboard,car)
                right=MovePossibleRight(gameboard,car)       
                if(left!=0 and fuelLevel[car]>left):
                    carMoves.update({"left":left})
                if(right!=0 and fuelLevel[car]>right):
                    carMoves.update({"right":right})

            if(axis=='y'):

                up=MovePossibleUp(gameboard,car)
                down=MovePossibleDown(gameboard,car)
                if(up!=0 and fuelLevel[car]>up):

                    carMoves.update({"up":up})
                if(down!=0 and fuelLevel[car]>down):
                    carMoves.update({"down":down})

            if(len(carMoves)>0):
                moves.update({car:carMoves})

    
    return moves

#return spaces car can move left
def MovePossibleLeft(gameboard,car):
    count=0
    index=gameboard.index(car)
    while(index%6 != 0):
        index-=1
        if(gameboard[index]=="."):
            count+=1
            continue
        else:
            break
    return count


#return spaces car can move right
def MovePossibleRight(gameboard,car):
    count=0
    index=gameboard.index(car)
    while(index%6 != 5):
        index+=1
        if(gameboard[index]=="."):
            count+=1
            continue
        if(gameboard[index]==car):
            continue
        else:
            break
    return count


#return spaces car can move Up
def MovePossibleUp(gameboard,car):
    count=0
    index=gameboard.index(car)-6
    while(index>=0):
        
        if(gameboard[index]=="."):
            count+=1
            index-=6
            continue
        else:
            break
    return count

#return spaces car can move down
def MovePossibleDown(gameboard,car):
    count=0
    index=gameboard.index(car)+6
    
    while(index<=35):
        if(gameboard[index]=="."):
            count+=1
            index+=6
            continue
        if(gameboard[index]==car):
            index+=6
            continue
        else:
            break
    return count
#return possible gameboard with possible moves based on a 6x6 gameboard and fuel dict
def possibleboard(gameboard, fuel):
    possiblemove=possiblemoves(gameboard,fuel)
    gameboards=[]
    gameboard=gameboard.flatten().tolist()
    
    for car in possiblemove:
        
        for direction in possiblemove[car]:
            count=1
            newBoard=gameboard.copy()
            newFuel=fuel.copy()
            index=newBoard.index(car)
            
            
            if direction=="up":
                while possiblemove[car][direction]>=count:
                    newBoard[index-6]=car
                    i=1
                   
                    while (newBoard[index+(i*6)]==car)  :
                        i+=1
                        if (index+i*6>35):
                            break
                    newBoard[index+(i*6)-6]='.'
                    option=np.array(newBoard).reshape(6,6)
                    newFuel[car]=newFuel[car]-1
                    gameboards.append((option,newFuel.copy()))
                    count+=1
                    index-=6
            if direction=="down":
                while possiblemove[car][direction]>=count:
                    newBoard[index]='.'
                    i=1
                    while(newBoard[index+(i*6)]==car) :
                        i+=1
                        if (index+i*6>35):
                            break
                    newBoard[index+(i*6)]=car
                    option=np.array(newBoard).reshape(6,6)
                    newFuel[car]=newFuel[car]-1
                    gameboards.append((option,newFuel.copy()))
                    count+=1
                    index+=6
            if direction== "left":
                while possiblemove[car][direction]>=count:
                    newBoard[index-1]='car'
                    i=1
                    while(newBoard[index]==car) :
                        i+=1
                        if (index-i)%6<5:
                            break
                    newBoard[index-i-1]='.'
                    option=np.array(newBoard).reshape(6,6)
                    newFuel[car]=newFuel[car]-1
                    gameboards.append((option,newFuel.copy()))
                    count+=1
                    index-=1
            if direction=="right":
                while possiblemove[car][direction]>=count:
                    newBoard[index]='.'
                    i=1
                    while (newBoard[index+(i)]==car) :
                        i+=1
                        if (index+i)%6>0:
                            break
                    newBoard[index+i]=car
                    valetService(newBoard)
                    option=np.array(newBoard).reshape(6,6)
                    newFuel[car]=newFuel[car]-1
                    gameboards.append((option,newFuel.copy()))
                    count+=1
                    index+=1
    return gameboards
            

In [129]:
%time
#test for functions

#test to return all possible moves based on a board layout and fuel level

for i in range(len(gameboards)):
    print(possibleboard(gameboards[i],fuel[i]))
    print()
       
#test for possiblemove function
for i in range(len(gameboards)):
    print("possible moves for initial board in test #", i)  
    print(possiblemoves(gameboards[i],fuel[i]))

CPU times: user 5 µs, sys: 25 µs, total: 30 µs
Wall time: 34.1 µs
[(array([['B', 'B', 'I', 'J', '.', '.'],
       ['.', '.', 'I', 'J', 'C', 'C'],
       ['.', '.', 'I', 'A', 'A', '.'],
       ['G', 'D', 'D', 'K', '.', 'M'],
       ['G', 'H', '.', 'K', 'L', 'M'],
       ['G', 'H', 'F', 'F', 'L', '.']], dtype='<U1'), {'B': 100, 'I': 100, 'J': 100, 'C': 100, 'A': 100, 'M': 99, 'G': 100, 'D': 100, 'K': 100, 'H': 100, 'L': 100, 'F': 100}), (array([['B', 'B', 'I', 'J', '.', '.'],
       ['.', '.', 'I', 'J', 'C', 'C'],
       ['.', '.', 'I', 'A', 'A', '.'],
       ['G', 'D', 'D', 'K', '.', '.'],
       ['G', 'H', '.', 'K', 'L', 'M'],
       ['G', 'H', 'F', 'F', 'L', 'M']], dtype='<U1'), {'B': 100, 'I': 100, 'J': 100, 'C': 100, 'A': 100, 'M': 98, 'G': 100, 'D': 100, 'K': 100, 'H': 100, 'L': 100, 'F': 100}), (array([['B', 'B', 'I', 'J', '.', '.'],
       ['.', '.', 'I', 'J', 'C', 'C'],
       ['G', '.', 'I', 'A', 'A', 'M'],
       ['G', 'D', 'D', 'K', '.', 'M'],
       ['G', 'H', '.', 'K', 'L',

In [83]:
#read the file "sample-input.txt"
gameboards,fuel=readfile("sample-input.txt")
#Below prints initial board and fuel level for all initial state in the txt file

#for i in range(len(gameboards)):
#    print("initial board configuration for board #" , i+1)
#    for j in range(len(gameboards[i])):
#       print(gameboards[i][j])
#   print("Car fuel available", fuel[i])


In [ ]:
from queue import priorityqueue
def Uniform Cost Search (gameboard,fuel):
    openq=priorityqueue()
    openq.put((0,gameboard,fuel))
    goalNode=none
    while(gameboard[17]!="A"):
        
        #g(n) (root to current node)
        #priorityqueue() sorted by g(n)
        possiblemoves(gameboard,fuel);
        openq=
        closedlist=[]
        valetService(gameboard);
    
def Greedy Best First Search (gameboard,fuel):
    while(gameboard[17]!="A"):
        #h(n) (estimate of lowest cost from node to goal)
        #priority queue sorted by h(n)
        possiblemoves(gameboard,fuel);
        valetService(gameboard);
    
def Algorithm A or A⋆(gameboard,fuel):
    while(gameboard[17]!="A"):
        #f(n)=h(n)+ g(n)
        #priority queue sorted by f(n)
        possiblemoves(gameboard,fuel);
        valetService(gameboard);

#returns the number of move done so far
def g(gameboard):

#return the estimate of moves left to reach the goal
def h(gameboard):
    